In [1]:
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data.dataset import Dataset 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms,models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
import shutil
from sklearn.metrics import f1_score   

c:\Users\rorod\miniforge3\envs\new_python_nlp_2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [3]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])

In [4]:
transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
train_dir = "Data/train/"
test_dir = "Data/test/"
val_dir= "Data/val/"
train_flooded_dir = "Data/train/flooded/"
train_non_flooded_dir = "Data/train/non-flooded/"
val_flooded_dir = "Data/val/flooded/"
val_non_flooded_dir = "Data/val/non-flooded/"
test_flooded_dir = "Data/test/flooded/"
test_non_flooded_dir = "Data/test/non-flooded/"

In [7]:
train_dataset = ImageFolder(train_dir, transforms_train)
test_dataset = ImageFolder(test_dir, transforms_test)
val_dataset = ImageFolder(val_dir, transforms_val)
train_dataloader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_dataset, batch_size=12, shuffle=True, num_workers=8)
test_dataloader = DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=8)

In [8]:
print('Train dataset size:', len(train_dataset))
print('Test dataset size:', len(test_dataset))
print('Val dataset size:', len(val_dataset))
class_names = train_dataset.classes
print('Class names:', class_names)

Train dataset size: 644
Test dataset size: 140
Val dataset size: 138
Class names: ['flooded', 'non-flooded']


In [9]:
model = models.regnet_x_1_6gf(pretrained=True)   #load vgg16 model


c:\Users\rorod\miniforge3\envs\new_python_nlp_2\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\rorod\miniforge3\envs\new_python_nlp_2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RegNet_X_1_6GF_Weights.IMAGENET1K_V1`. You can also use `weights=RegNet_X_1_6GF_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
num_features = model.fc.in_features     #extract fc layers features


In [11]:
model.fc = nn.Linear(num_features, 2) #(num_of_class == 2)


In [12]:
model = model.to(device) 


In [13]:
criterion = nn.CrossEntropyLoss()  #(set loss function)


In [14]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [15]:
num_epochs = 10   #(set no of epochs)
start_time = time.time() #(for showing time)

for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    targets = []
    results = []
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        results.append(preds)
        targets.append(labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    results =torch.cat(results, dim=0)
    targets = torch.cat(targets, dim=0)

    results = results.to('cpu').numpy().flatten()
    targets = targets.to('cpu').numpy().flatten()

    f1_value =  f1_score(results, targets)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))
    print('F1-Score', f1_value)
    """ Val Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        targets = []
        results = []
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            results.append(preds)
            targets.append(labels)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects / len(val_dataset) * 100.
        results = torch.cat(results, dim=0)
        targets =torch.cat(targets, dim=0)
        results = results.to('cpu').numpy().flatten()
        targets = targets.to('cpu').numpy().flatten()
        f1_value =  f1_score(results, targets)
        print('[Val #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))
        print('F1-Score', f1_value)

Epoch 0 running
[Train #0] Loss: 0.4875 Acc: 75.1553% Time: 103.8501s
F1-Score 0.7553516819571865
[Val #0] Loss: 0.1828 Acc: 97.1014% Time: 115.2358s
F1-Score 0.9714285714285714
Epoch 1 running
[Train #1] Loss: 0.2676 Acc: 88.6646% Time: 218.2594s
F1-Score 0.8864696734059097
[Val #1] Loss: 0.1041 Acc: 98.5507% Time: 228.8018s
F1-Score 0.9857142857142858
Epoch 2 running
[Train #2] Loss: 0.2162 Acc: 91.3044% Time: 331.9037s
F1-Score 0.9125000000000001
[Val #2] Loss: 0.0773 Acc: 98.5507% Time: 342.1626s
F1-Score 0.9855072463768116
Epoch 3 running
[Train #3] Loss: 0.1381 Acc: 95.3416% Time: 445.6123s
F1-Score 0.9539877300613496
[Val #3] Loss: 0.0583 Acc: 98.5507% Time: 456.4333s
F1-Score 0.9855072463768116
Epoch 4 running
[Train #4] Loss: 0.1654 Acc: 93.9441% Time: 559.8204s
F1-Score 0.9402756508422665
[Val #4] Loss: 0.0663 Acc: 98.5507% Time: 570.6444s
F1-Score 0.9855072463768116
Epoch 5 running
[Train #5] Loss: 0.1266 Acc: 95.0311% Time: 676.1151s
F1-Score 0.9503105590062112
[Val #5] Los

In [16]:
model.eval()
with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    targets = []
    results = []
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        results.append(preds)
        targets.append(labels)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects / len(test_dataset) * 100.
    results = torch.cat(results, dim=0)
    targets =torch.cat(targets, dim=0)
    results = results.to('cpu').numpy().flatten()
    targets = targets.to('cpu').numpy().flatten()
    f1_value =  f1_score(results, targets)
    print('F1-Score', f1_value)
    print('[Test ] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format( epoch_loss, epoch_acc, time.time()- start_time))

F1-Score 1.0
[Test ] Loss: 0.0418 Acc: 100.0000% Time: 1152.3409s
